In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Dec 21 18:35:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%capture
!pip install datasets==1.13.3
!pip install transformers==4.11.3
!pip install huggingface_hub==0.1
!pip install torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install jiwer

In [ ]:
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        
Username: akashsivanandan
Password: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-crendential store but this isn't the helper defined on your machine.
You will have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal to set it as the default

git config --global credential.helper store


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

401 Client Error: Unauthorized for url: https://huggingface.co/api/login
Unauthorized


In [ ]:
%%capture
!apt install git-lfs

##Creating Wave2VecCTCTokenizer

In [ ]:
from datasets import load_dataset, load_metric, Audio

common_voice_train = load_dataset("common_voice", "ta", split="train+validation")
common_voice_test = load_dataset("common_voice", "ta", split="test")

Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/ta/6.1.0/f7a9d973839b7706e9e281c19b7e512f31badf3c0fdbd21c671f3c4bf9acf3b9)
Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/ta/6.1.0/f7a9d973839b7706e9e281c19b7e512f31badf3c0fdbd21c671f3c4bf9acf3b9)


In [ ]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(common_voice_train.remove_columns(["path", "audio"]), num_examples=10)

,sentence
0,சொற் சோர்வு படேல்.
1,கேண்மைகொள் சேசுவின் கீர்த்தி யுரைத்திடு தோழி - அவர்
2,இங்கும் அடங்கிடும் பாடிய பறவைகள்!
3,வீட்டின் தலைவர் விளக்கேற்று தற்கு
4,பொன்நிறக் கதிரொடு போந்த பகலவன்
5,கடல்போலச் செந்தமிழைப் பெருக்க வேண்டும்!
6,தனம் தேடி உண்ணாமல் புதைக்க வேண்டாம்
7,வழியென்று ஜாதியென் றேயுரைப் பார்கள்;
8,கத்தி முனைகாட்டிக் காலமெல்லாம் ஏய்த்துவரும்
9,சஞ்சீவி பர்வதத்தில் கூப்பிட்டேன். தற்செயலாய்


In [ ]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\'\।]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()
    return batch

In [ ]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

  0%|          | 0/3788 [00:00<?, ?ex/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ta/6.1.0/f7a9d973839b7706e9e281c19b7e512f31badf3c0fdbd21c671f3c4bf9acf3b9/cache-0cc52dd73997372d.arrow


In [ ]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]))

,sentence
0,இதன் இனப்பெருக்கக் காலம் இடத்துக்கு ஏற்ப மாறுபடும்
1,ஆனினம் சுமந்தமடி
2,அல்லவை தேய் அறம்பெருகும் நல்லவை நாடி இனிய சொலின்
3,ராசபுத் தானத்து வீரர் தமக்கு
4,சொர்ணம் வந்தால் கொஞ்ச நேரம்மட்டும்
5,நாயிலுங் கடையாய் நலிவது மேலா
6,புல்லனை நான்மணக்க
7,மறமென்ற செயல்என்றும் மதியாத மொழியாம்
8,பூச்செடிகளை நட்டு வைக்க போகிறோம்
9,பெற்றதாய் மாரேநல் இளஞ்சிங் கங்காள்


In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

{' ': 0,
 'ஃ': 1,
 'அ': 2,
 'ஆ': 3,
 'இ': 4,
 'ஈ': 5,
 'உ': 6,
 'ஊ': 7,
 'எ': 8,
 'ஏ': 9,
 'ஐ': 10,
 'ஒ': 11,
 'ஓ': 12,
 'ஔ': 13,
 'க': 14,
 'ங': 15,
 'ச': 16,
 'ஜ': 17,
 'ஞ': 18,
 'ட': 19,
 'ண': 20,
 'த': 21,
 'ந': 22,
 'ன': 23,
 'ப': 24,
 'ம': 25,
 'ய': 26,
 'ர': 27,
 'ற': 28,
 'ல': 29,
 'ள': 30,
 'ழ': 31,
 'வ': 32,
 'ஷ': 33,
 'ஸ': 34,
 'ஹ': 35,
 'ா': 36,
 'ி': 37,
 'ீ': 38,
 'ு': 39,
 'ூ': 40,
 'ெ': 41,
 'ே': 42,
 'ை': 43,
 'ொ': 44,
 'ோ': 45,
 'ௌ': 46,
 '்': 47,
 '’': 48}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

51

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
repo_name = "wav2vec2-large-xls-r-300m-tamil-colab-final"

In [ ]:
tokenizer.push_to_hub(repo_name)

OSError: ignored

##Creating a feature extractor

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

##Preprocessing Data

In [ ]:
common_voice_train[0]["path"]

'/root/.cache/huggingface/datasets/downloads/extracted/38a26df4350f0cbefb5a50d4018959b95a7ee56e372bfbcd98491bc12d6599c7/cv-corpus-6.1-2020-12-11/ta/clips/common_voice_ta_19171502.mp3'

In [ ]:
common_voice_train[0]["audio"]

{'array': array([0.        , 0.        , 0.        , ..., 0.00075823, 0.00074035,
        0.00111735], dtype=float32),
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/38a26df4350f0cbefb5a50d4018959b95a7ee56e372bfbcd98491bc12d6599c7/cv-corpus-6.1-2020-12-11/ta/clips/common_voice_ta_19171502.mp3',
 'sampling_rate': 48000}

In [ ]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
common_voice_train[0]["audio"]

{'array': array([0.        , 0.        , 0.        , ..., 0.00103243, 0.00046855,
        0.00098617], dtype=float32),
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/38a26df4350f0cbefb5a50d4018959b95a7ee56e372bfbcd98491bc12d6599c7/cv-corpus-6.1-2020-12-11/ta/clips/common_voice_ta_19171502.mp3',
 'sampling_rate': 16000}

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

print(common_voice_train[rand_int]["sentence"])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

துணைபிடிப்பாய்  பயம்விடுப்பாய்


In [ ]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["sentence"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

Target text: விழிபெற்றுப் பதவி கொள்வார்
Input array shape: (54912,)
Sampling rate: 16000


In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

Let's apply the data preparation function to all examples.

In [ ]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)

  0%|          | 0/3788 [00:00<?, ?ex/s]

  0%|          | 0/1781 [00:00<?, ?ex/s]

In [ ]:
#max_input_length_in_sec = 5.0
#common_voice_train = common_voice_train.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

##Training

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")

Downloading:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m", 
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

Downloading:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-xls-r-300m were not used when initializing Wav2Vec2ForCTC: ['quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'project_q.bias', 'project_hid.weight', 'project_hid.bias', 'quantizer.codevectors', 'project_q.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it 

In [ ]:
model.freeze_feature_extractor()

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="epoch",
  logging_strategy="epoch",
  num_train_epochs=30,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:718: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
/content/wav2vec2-large-xls-r-300m-tamil-colab-final is already a clone of https://huggingface.co/akashsivanandan/wav2vec2-large-xls-r-300m-tamil-colab-final. Make sure you pull the latest changes with `repo.git_pull()`.
Using amp fp16 backend


```javascript
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);
```

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running training *****
  Num examples = 3788
  Num Epochs = 30
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 3540
/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:882: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like t

Epoch,Training Loss,Validation Loss,Wer
0,11.146600,4.344378,1.000000
1,3.418800,3.249573,1.000000
2,2.861700,1.616537,1.000281
3,0.958000,0.798431,0.871997
4,0.592900,0.673263,0.783055
5,0.462800,0.653586,0.762119
6,0.383400,0.603730,0.715470
7,0.324200,0.637566,0.718421
8,0.273600,0.621354,0.707039
9,0.243300,0.615848,0.694394


The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running Evaluation *****
  Num examples = 1781
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running Evaluation *****
  Num examples = 1781
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running Evaluation *****
  Num examples = 1781
  Batch size = 8
Saving model checkpoint to wav2vec2-large-xls-r-300m-tamil-colab-final/checkpoint-400
Configuration saved in wav2vec2-large-xls-r-300m-tamil-colab-final/checkpoint-400/config.json
Model weights saved in wav2vec2-large-xls-r-300m-tamil-colab-final/checkpoint-400/pytorch_model.bin
Configuration saved in wav2vec2-large-xls-r-300m-tamil-colab-final/check

TrainOutput(global_step=3540, training_loss=0.7670345652574874, metrics={'train_runtime': 23362.4942, 'train_samples_per_second': 4.864, 'train_steps_per_second': 0.152, 'total_flos': 1.507206976721773e+19, 'train_loss': 0.7670345652574874, 'epoch': 30.0})

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to wav2vec2-large-xls-r-300m-tamil-colab-final
Configuration saved in wav2vec2-large-xls-r-300m-tamil-colab-final/config.json
Model weights saved in wav2vec2-large-xls-r-300m-tamil-colab-final/pytorch_model.bin
Configuration saved in wav2vec2-large-xls-r-300m-tamil-colab-final/preprocessor_config.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.35k/1.18G [00:00<?, ?B/s]

Upload file runs/Dec21_18-38-57_89d9f9d676d1/events.out.tfevents.1640112043.89d9f9d676d1.83.0:  18%|#8        …

To https://huggingface.co/akashsivanandan/wav2vec2-large-xls-r-300m-tamil-colab-final
   a305f06..e6ef0cc  main -> main

Dropping the following result as it does not have all the necessary field:
{'dataset': {'name': 'common_voice', 'type': 'common_voice', 'args': 'ta'}}
To https://huggingface.co/akashsivanandan/wav2vec2-large-xls-r-300m-tamil-colab-final
   e6ef0cc..de36f49  main -> main



'https://huggingface.co/akashsivanandan/wav2vec2-large-xls-r-300m-tamil-colab-final/commit/e6ef0cc5cf5b934d18eb49bce29d5d000363de8e'

## Evaluation

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained(repo_name).to("cuda")
processor = Wav2Vec2Processor.from_pretrained(repo_name)

loading configuration file wav2vec2-large-xls-r-300m-tamil-colab-final/config.json
Model config Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-xls-r-300m",
  "activation_dropout": 0.0,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 768,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "feat_extract_norm": "layer",
  "feat_proj_dropout": 0.0,
  "feat_quantizer_dropout": 0.0,
  "final_dropout": 0.0,
  "

In [ ]:
input_dict = processor(common_voice_test[0]["input_values"], return_tensors="pt", padding=True)

logits = model(input_dict.input_values.to("cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)[0]

It is strongly recommended to pass the ``sampling_rate`` argument to this function.Failing to do so can result in silent errors that might be hard to debug.


In [ ]:
common_voice_test_transcription = load_dataset("common_voice", "ta", data_dir="./cv-corpus-6.1-2020-12-11", split="test")

Using custom data configuration ta-ad9f7b76efa9f3a0


0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset common_voice downloaded and prepared to /root/.cache/huggingface/datasets/common_voice/ta-ad9f7b76efa9f3a0/6.1.0/f7a9d973839b7706e9e281c19b7e512f31badf3c0fdbd21c671f3c4bf9acf3b9. Subsequent calls will reuse this data.


In [ ]:
print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(common_voice_test_transcription[0]["sentence"].lower())

Prediction:
தாயும் பிள்ளையும் சரலின் நிருுந்தே

Reference:
தாயும் பிள்ளையும் சரேலென் றெழுந்தே
